In [ ]:
import os
import sys
import subprocess

print("⬇️ AVVIO SETUP AMBIENTE PER YOLOV8...")

# Only install the official library
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "ultralytics"])

# Remove “ray”, which often causes problems with final logs on Kaggle
subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "ray"])

print("✅ Setup completato. Ambiente pronto per YOLOv8.")

In [ ]:
%%writefile train.py
import os
import glob
import random
import yaml
import torch
import gc
import sys
import numpy
from ultralytics import YOLO
from ultralytics import settings

try:
    # Configuration
    EXPERIMENT = '50pct' #100pct, 25pct
    EPOCHS = 100
    BATCH_SIZE = 16

    # Setup WANDB
    os.environ['WANDB_MODE'] = 'disabled'
    settings.update({"raytune": False})

    # Dataset management
    print("🔍 Verifica Dataset...", flush=True)
    dataset_path = ''
    potential_paths = [
        './datasets/VisDrone',
        '/kaggle/input/visdrone/VisDrone',
        '/kaggle/working/datasets/VisDrone'
    ]

    # Download dummy if needed
    if not any(os.path.exists(p) for p in potential_paths):
        try:
            model_dummy = YOLO('yolov8s.pt') 
            model_dummy.train(data='VisDrone.yaml', epochs=1, imgsz=32, batch=1)
            del model_dummy
            gc.collect()
            torch.cuda.empty_cache()
        except Exception as e:
            print(f" Warning nel download dummy: {e}")

    for path in potential_paths:
        if os.path.exists(path):
            dataset_path = path
            break
    if not dataset_path:
        dataset_path = '/kaggle/working/datasets/VisDrone'
    print(f" Dataset path: {dataset_path}", flush=True)

    # Data preparation
    path_a_train = os.path.join(dataset_path, 'VisDrone2019-DET-train/images/*.jpg')
    path_b_train = os.path.join(dataset_path, 'images/train/*.jpg')
    
    path_a_val = os.path.join(dataset_path, 'VisDrone2019-DET-val/images/*.jpg')
    path_b_val = os.path.join(dataset_path, 'images/val/*.jpg')

    train_images = glob.glob(path_a_train)
    val_images = glob.glob(path_a_val)

    if not train_images:
        train_images = glob.glob(path_b_train)
        val_images = glob.glob(path_b_val)

    if not train_images:
        raise FileNotFoundError(f" NESSUNA IMMAGINE TROVATA IN: {dataset_path}")

    if not val_images:
        print(" ATTENZIONE: Nessuna immagine di validazione trovata! Uso fallback.", flush=True)
        val_images = train_images[:100]

    random.seed(42)
    random.shuffle(train_images)
    selected_images = train_images 

    print(f" Training su {EXPERIMENT}: {len(selected_images)} immagini.", flush=True)

    cwd = os.getcwd()
    
    visdrone_classes = {
        0: 'pedestrian', 1: 'people', 2: 'bicycle', 3: 'car', 4: 'van',
        5: 'truck', 6: 'tricycle', 7: 'awning-tricycle', 8: 'bus', 9: 'motor'
    }

    txt_filename = os.path.join(cwd, f'train_{EXPERIMENT}.txt')
    with open(txt_filename, 'w') as f:
        f.write('\n'.join(selected_images))
    
    val_txt = os.path.join(cwd, 'val_official.txt')
    with open(val_txt, 'w') as f:
        f.write('\n'.join(val_images))

    yaml_content = {
        'path': cwd,
        'train': txt_filename,
        'val': val_txt,
        'names': visdrone_classes
    }
    
    yaml_filename = os.path.join(cwd, f'VisDrone_{EXPERIMENT}.yaml')
    with open(yaml_filename, 'w') as f:
        yaml.dump(yaml_content, f) 
    
    print(f" File YAML scritto correttamente: {yaml_filename}", flush=True)

    # Training
    print(f"\n{'='*20} AVVIO TRAINING: {EXPERIMENT} {'='*20}", flush=True)
    
    model = YOLO('yolov8s.pt')

    model.train(
        data=yaml_filename,
        epochs=EPOCHS,
        imgsz=640,
        batch=BATCH_SIZE,
        device=0,
        optimizer='AdamW',
        lr0=0.001,
        freeze=10,
        patience=10,
        project='YOLOv8_VisDrone',
        name=f'run_{EXPERIMENT}',
        exist_ok=True,
        verbose=True
    )

except Exception as e:
    print(f"\n ERRORE CRITICO:\n{e}", flush=True)
    import traceback
    traceback.print_exc()
    sys.exit(1)

In [ ]:
!python train.py

import os
import shutil

# Delete the dataset to prevent it from being added to the output zip file.
if os.path.exists("./datasets"):
    shutil.rmtree("./datasets")
    print(" Cartella dataset eliminata.")

exp_name = "100pct" #50pct, 25pct
output_zip = f"risultati_yolo_VisDrone_{exp_name}.zip"

folders_to_zip = f"YOLOv8_VisDrone/run_{exp_name} runs/detect/train"
os.system(f"zip -r -q {output_zip} {folders_to_zip}")

if os.path.exists(output_zip):
    print(f" Archivio pronto: {output_zip}")
else:
    print(" Attenzione: Archivio non creato.")